In [ ]:
!gdown 17JHIac30dyF3P5BRS_FIvUbJc04lQJWC

Downloading...
From: https://drive.google.com/uc?id=17JHIac30dyF3P5BRS_FIvUbJc04lQJWC
To: /content/Nutrition_Training_Data.csv
100% 4.38M/4.38M [00:00<00:00, 30.5MB/s]


In [ ]:
!pip install paddleocr
!pip install paddlepaddle
!git clone https://github.com/PaddlePaddle/PaddleOCR

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 377 kB 7.6 MB/s 
     |████████████████████████████████| 4.9 MB 65.6 MB/s 
     |████████████████████████████████| 2.3 MB 56.0 MB/s 
     |████████████████████████████████| 604 kB 103.3 MB/s 
     |████████████████████████████████| 399 kB 78.4 MB/s 
     |████████████████████████████████| 104 kB 77.9 MB/s 
     |████████████████████████████████| 204 kB 93.2 MB/s 
     |████████████████████████████████| 2.3 MB 52.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 112.5 MB 62 kB/s 
     |████████████████████████████████| 394 kB 83.9 MB/s 
Cloning into 'PaddleOCR'...
remote: Enumerating objects: 42470, done.
remote: Counting objects: 100% (710/710), done.
remote: Compressing objects: 100% (368/368), done.
remote: Total 42470 (delta 400), reused 593 (delta 33

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from numpy import argmax
from sklearn.metrics import mean_absolute_error
import numpy as np
import re
from skimage import io
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import pandas as pd
from paddleocr import PPStructure,save_structure_res
import os
import cv2

In [ ]:
table_engine = PPStructure(layout=False, show_log=True)

download https://paddleocr.bj.bcebos.com/PP-OCRv3/chinese/ch_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer/ch_PP-OCRv3_det_infer.tar


100%|██████████| 3.83M/3.83M [00:15<00:00, 253kiB/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv3/chinese/ch_PP-OCRv3_rec_infer.tar to /root/.paddleocr/whl/rec/ch/ch_PP-OCRv3_rec_infer/ch_PP-OCRv3_rec_infer.tar


100%|██████████| 11.9M/11.9M [00:11<00:00, 1.01MiB/s]


download https://paddleocr.bj.bcebos.com/ppstructure/models/slanet/ch_ppstructure_mobile_v2.0_SLANet_infer.tar to /root/.paddleocr/whl/table/ch_ppstructure_mobile_v2.0_SLANet_infer/ch_ppstructure_mobile_v2.0_SLANet_infer.tar


100%|██████████| 10.3M/10.3M [00:26<00:00, 393kiB/s] 


download https://paddleocr.bj.bcebos.com/ppstructure/models/layout/picodet_lcnet_x1_0_fgd_layout_cdla_infer.tar to /root/.paddleocr/whl/layout/picodet_lcnet_x1_0_fgd_layout_cdla_infer/picodet_lcnet_x1_0_fgd_layout_cdla_infer.tar


100%|██████████| 10.1M/10.1M [00:10<00:00, 979kiB/s] 


[2022/09/06 12:04:10] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir=None, cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/root/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thresh=0.5, e2e_pgnet_valid_set

In [ ]:
df = pd.read_csv('/content/Nutrition_Training_Data.csv')

In [ ]:
df.head(10)

,ingredients_text,allergens,image_ingredients_url,image_nutrition_url,fat_100g,saturated_fat_100g,sugars_100g,salt_100g,nutriscore_score,nutriscore_grade
0,"SUGAR, CORN SYRUP SOLIDS, VEGETABLE OIL (PARTI...","en:milk,en:soybeans,en:COCONUT",https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,10.0,7.5,60.0,2.3750,30.0,e
1,"peanuts, dextrose hydrogenated vegetable oil (...",en:peanuts,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,280.0,200.0,60.0,32.0000,40.0,e
2,Cooked Chicken Breast (26%) Cooked Pilau Rice ...,en:milk,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,3.0,1.0,1.2,0.4500,-7.0,a
3,"Azucar (Maltodextrina), Cacao en polvo, Emulsi...",en:soybeans,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,0.0,0.0,80.0,0.0375,31.0,e
4,"_Almond flour_*, golden linseed flour*, pumpki...","en:nuts,en:Almond flour,en:egg white powder",https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,1.8,0.4,1.2,0.3800,-8.0,a
5,"Wholegrain oats 75%, apples 10,8%, date powder...",en:gluten,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,5.8,1.0,16.5,0.0100,-8.0,a
6,"pearl _barley_ (25%), haricot beans (14%), yel...",en:gluten,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,1.0,0.2,0.2,0.0300,-8.0,a
7,"Chickpeas, Water, Antioxidant (_Sodium Metabis...",en:Sodium Metabisulphite,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,2.2,0.3,0.0,0.0200,-8.0,a
8,"Butter Beans 60%, Water, Firming Agent (Calciu...",en:sulphur-dioxide-and-sulphites,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,0.5,0.0,0.0,0.0500,-13.0,a
9,"potato (97%), sunflower oil, fortified wheat f...",en:gluten,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,2.0,0.2,0.4,0.1900,-8.0,a


In [ ]:
df.shape

(8053, 10)

In [ ]:
df.isna().sum()

ingredients_text            0
allergens                   0
image_ingredients_url       0
image_nutrition_url         0
fat_100g                  532
saturated_fat_100g        614
sugars_100g               629
salt_100g                 639
nutriscore_score         1559
nutriscore_grade         1559
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.shape

(6476, 10)

In [ ]:
def cleanhtml(raw_html):
  cleantext = re.sub('<.*?>', ' ', raw_html)
  return cleantext

def change_to_g(text):
    search_ln = re.search("\d\s|\d$", text)
    if search_ln and search_ln.group().strip() == "9":
        index = search_ln.span()[0]
        text = text[:index] +"g"+ text[index+1:]

    search_lnq = re.search("\dmq\s|\dmq$", text)
    if search_lnq:
        index = search_lnq.span()[0] +2
        text = text[:index] +"g"+ text[index+1:]
    return text

def clean_string(string):
    pattern = "[\|\*\_\'\—\-\{}]".format('"')
    text = re.sub(pattern, "", string)
    text = re.sub(" I ", " / ", text)
    text = re.sub("^I ", "", text)
    text = re.sub("Omg", "0mg", text)
    text = re.sub("Og", "0g", text)
    text = re.sub('(?<=\d) (?=\w)', '', text)
    text = change_to_g(text)
    text = text.strip()
    return text

In [ ]:
image = io.imread(df.image_nutrition_url[5])
result = table_engine(image)
temp = result[0]['res']['html']

[2022/09/05 18:43:38] ppocr DEBUG: dt_boxes num : 30, elapse : 0.13033175468444824
[2022/09/05 18:43:40] ppocr DEBUG: rec_res num  : 30, elapse : 2.8025543689727783


In [ ]:
def check_for_label(text, words):
    # text = text.lower()
    for i in range(len(text)):
        if any(text[i:].startswith(word) for word in words):
            return True
    return False

check_for_label(temp, 'Salt')

True

In [ ]:
def get_label_from_string(string):
    label_arr = re.findall("([A-Z][a-zA-Z]*)", string)
    label_name = ""
    label_value = ""

    if len(label_arr) == 0:
        label_name = "|"+string+'|'
    elif len(label_arr) == 1:
        label_name = label_arr[0]
    else:
        label_name = label_arr[0] + ' ' + label_arr[1]

    digit_pattern = "[-+]?\d*\.\d+g|\d+"
    value_arr = re.findall("{0}g|{0}%|{0}J|{0}kJ|{0}mg|{0}kcal".format(digit_pattern), string)
    # print(value_arr)
    if len(value_arr):
        label_value = value_arr[0]
    else:
        label_value = "|"+string+'|'
    return label_name, label_value

In [ ]:
get_label_from_string(temp)

('NUTRITIONAL INFORMATION', '17g')

In [ ]:
temp

'NUTRITIONAL INFORMATION Serves per package:1 Quantity Quantity Unnamed: 1 Unnamed: 2 Serving size:500mL Energy per serving 1650kj 17g per100mL 330KProtein Fat-Total -Saturated Carbohydrate -Total Sugars* -Lactose Sodium Caffeine *Totalsugarsincludelactose. 18.5g 12.5g 40g 36g 20.5g 160mg 180mg 3.4g 3.7g 2.5g 8g 7.2g 4.1g 32mg 36mg'

In [ ]:
save_folder = './output/'
img_path = df.image_nutrition_url[40]
img = io.imread(df.image_nutrition_url[40])
result = table_engine(img)
save_structure_res(result, save_folder, os.path.basename(img_path).split('.')[0])
path = save_folder+os.path.basename(img_path).split('.')[0]
filename = os.listdir(path)[0]
ex = pd.read_excel(path+'/'+filename)
text=ex.to_string(index=False).replace("\n", "")
text=" ".join(text.split())
shutil.rmtree(path, ignore_errors=True)
temp = change_to_g(text)

[2022/09/06 12:27:02] ppocr DEBUG: dt_boxes num : 35, elapse : 0.11729192733764648
[2022/09/06 12:27:05] ppocr DEBUG: rec_res num  : 35, elapse : 2.9282584190368652


In [ ]:
import shutil

In [ ]:
table_engine = PPStructure(layout=False, show_log=True)


[2022/09/06 12:17:09] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir=None, cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/root/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thresh=0.5, e2e_pgnet_valid_set

In [ ]:
def extract_nutrients(image_url):
  save_folder = './output/'
  img_path = image_url
  img = io.imread(image_url)
  result = table_engine(img)
  save_structure_res(result, save_folder, os.path.basename(img_path).split('.')[0])
  path = save_folder+os.path.basename(img_path).split('.')[0]
  filename = os.listdir(path)[0]
  ex = pd.read_excel(path+'/'+filename)
  text=ex.to_string(index=False).replace("\n", "")
  text=" ".join(text.split())
  shutil.rmtree(path, ignore_errors=True)
  temp = change_to_g(text)
  fat = re.search('Fat (.+?)g', temp, re.IGNORECASE)
  if fat:
    int_fat = fat.group(1)
  else:
    int_fat = 0

  sat_fat = re.search('of which Saturates (.+?)g', temp, re.IGNORECASE)
  if sat_fat:
    int_sat_fat = sat_fat.group(1)
  else:
    int_sat_fat = 0

  sugar = re.search('Sugars (.+?)g', temp, re.IGNORECASE)
  if sugar:
    int_sugar = sugar.group(1)
  else:
    int_sugar = 0
  
  salt = re.search('Salt (.+?)g', temp, re.IGNORECASE)
  if salt:
    int_salt = salt.group(1)
  else:
    int_salt = 0

 
  return int_fat, int_sat_fat, int_sugar, int_salt


In [ ]:
fat, sat_fat, sugar, salt= extract_nutrients(df.image_nutrition_url[40])

[2022/09/06 12:24:50] ppocr DEBUG: dt_boxes num : 35, elapse : 0.11949777603149414
[2022/09/06 12:24:53] ppocr DEBUG: rec_res num  : 35, elapse : 2.904879331588745


In [ ]:
df.image_nutrition_url[40]

'https://images.openfoodfacts.org/images/products/933/753/600/0565/nutrition_en.17.400.jpg'

In [ ]:
print('Fats', fat)
print('Saturated fats', sat_fat)
print('Sugar', sugar)
print('Salt', salt)

Fats 0
Saturated fats 0
Sugar 0
Salt 0


In [ ]:
X = df.drop(['ingredients_text', 'allergens','image_ingredients_url',	'image_nutrition_url', 'nutriscore_score','nutriscore_grade'], axis=1)

In [ ]:
X.head()

,fat_100g,saturated_fat_100g,sugars_100g,salt_100g
0,10.0,7.5,60.0,2.3750
1,280.0,200.0,60.0,32.0000
2,3.0,1.0,1.2,0.4500
3,0.0,0.0,80.0,0.0375
4,1.8,0.4,1.2,0.3800


In [ ]:
y_score = df.nutriscore_score
y_grade =	df.nutriscore_grade
classes = len(list(y_grade.unique()))
le = LabelEncoder()
y_grade = le.fit_transform(y_grade)
scaler = StandardScaler()
X = scaler.fit_transform(X)
n_features=X.shape[1]

In [ ]:
X_train, X_test, y_train_score, y_test_score, y_train_grade, y_test_grade = train_test_split(X, y_score, y_grade, test_size=0.33, random_state=1)

In [ ]:
input = Input(shape=(n_features,))
x = Dense(400, activation='relu', kernel_initializer='he_normal')(input)
x = Dense(400, activation='relu', kernel_initializer='he_normal')(x)
x = Dense(400, activation='relu', kernel_initializer='he_normal')(x)
x = Dense(400, activation='relu', kernel_initializer='he_normal')(x)
score_out = Dense(1, activation='linear')(x)
grade_out = Dense(classes, activation='softmax')(x)
model = Model(inputs=input, outputs=[score_out, grade_out])
model.compile(loss=['mse','sparse_categorical_crossentropy'], optimizer='adam')
plot_model(model, to_file='model.png')
model.fit(X_train, [y_train_score,y_train_grade], epochs=150, batch_size=32, verbose=2)

Epoch 1/150
136/136 - 2s - loss: 86.6621 - dense_4_loss: 85.4274 - dense_5_loss: 1.2348 - 2s/epoch - 16ms/step
Epoch 2/150
136/136 - 1s - loss: 74.0738 - dense_4_loss: 73.0218 - dense_5_loss: 1.0520 - 1s/epoch - 9ms/step
Epoch 3/150
136/136 - 1s - loss: 61.1882 - dense_4_loss: 60.1603 - dense_5_loss: 1.0279 - 1s/epoch - 10ms/step
Epoch 4/150
136/136 - 1s - loss: 41.8380 - dense_4_loss: 40.8207 - dense_5_loss: 1.0173 - 1s/epoch - 8ms/step
Epoch 5/150
136/136 - 1s - loss: 22.3607 - dense_4_loss: 21.4316 - dense_5_loss: 0.9291 - 1s/epoch - 8ms/step
Epoch 6/150
136/136 - 1s - loss: 24.4237 - dense_4_loss: 23.5318 - dense_5_loss: 0.8919 - 1s/epoch - 8ms/step
Epoch 7/150
136/136 - 1s - loss: 32.6612 - dense_4_loss: 31.7591 - dense_5_loss: 0.9021 - 1s/epoch - 8ms/step
Epoch 8/150
136/136 - 1s - loss: 17.9707 - dense_4_loss: 17.1078 - dense_5_loss: 0.8629 - 1s/epoch - 8ms/step
Epoch 9/150
136/136 - 1s - loss: 18.2736 - dense_4_loss: 17.4171 - dense_5_loss: 0.8565 - 993ms/epoch - 7ms/step
Epoch

In [ ]:
score, grade = model.predict(X_test)
# calculate error for regression model
error = mean_absolute_error(y_test_score, score)
print('MAE: %.3f' % error)
# evaluate accuracy for classification model
grade = argmax(grade, axis=-1).astype('int')
acc = accuracy_score(y_test_grade, grade)
print('Accuracy: %.3f' % acc)

MAE: 2.148
Accuracy: 0.705


In [ ]:
model.save('model.h5')

In [ ]:
loaded_model = load_model('model.h5')

In [ ]:
nutrients = [fat, sat_fat, sugar, salt]
n = scaler.fit_transform(np.array(nutrients).reshape(1, -1))
score, grade = loaded_model.predict(n)
grade = argmax(grade, axis=-1).astype('int')
print('Nutrition SCORE', score)
print('Nutrition GRADE', le.inverse_transform([grade]))

Nutrition SCORE [[18.12929]]
Nutrition GRADE ['d']
